In [1]:
import pickle
import numpy
import pandas
import keras

Using TensorFlow backend.


In [2]:
with open('Data_Spectrograms.pkl', 'rb') as f:
    X, y = pickle.load(f)

In [3]:
from sklearn.model_selection import train_test_split
y = keras.utils.to_categorical(y, num_classes=6)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

In [4]:
X_train_a, X_train_b = numpy.split(X_train, 2, axis=1)
X_test_a, X_test_b = numpy.split(X_test, 2, axis=1)

In [5]:
X_train_a = numpy.squeeze(X_train_a)
X_train_b = numpy.squeeze(X_train_b)
X_test_a = numpy.squeeze(X_test_a)
X_test_b = numpy.squeeze(X_test_b)

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, concatenate
from keras.optimizers import Adam

def build_model():
    model = Sequential()
    model.add(Conv1D(32, kernel_size=4, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01),
                     input_shape=(100,30)))
    model.add(MaxPooling1D(2,2))
    model.add(Conv1D(32, kernel_size=4, kernel_regularizer=keras.regularizers.l2(0.01), activation='relu'))
    model.add(Conv1D(32, kernel_size=4, kernel_regularizer=keras.regularizers.l2(0.01), activation='relu'))
    model.add(MaxPooling1D(2,2))
    model.add(Conv1D(32, kernel_size=4, activation='relu'))
    model.add(MaxPooling1D(2,2))
    model.add(Conv1D(32, kernel_size=4, activation='relu'))
    model.add(Flatten())
    return model

In [7]:
model_a = build_model()
model_b = build_model()
combined_output = concatenate([model_a.output, model_b.output])
merged_layers = Dense(128, activation='relu')(combined_output)
merged_layers = Dense(6, activation='softmax')(merged_layers)
model = Model(inputs=[model_a.input, model_b.input], outputs=merged_layers)

In [8]:
model.compile(optimizer=Adam(lr=0.001, decay=0.001/80), loss='categorical_crossentropy', metrics=['acc'])

In [9]:
trained_model = model.fit([X_train_a, X_train_b], y_train,
                          validation_data=([X_test_a, X_test_b], y_test),
                          epochs=80, batch_size=32)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 13837 samples, validate on 1538 samples
Epoch 1/80
13837/13837 [==============================] - 7s 484us/step - loss: 2.5807 - acc: 0.6041 - val_loss: 1.7523 - val_acc: 0.7048
Epoch 2/80
13837/13837 [==============================] - 5s 380us/step - loss: 1.5876 - acc: 0.7062 - val_loss: 1.3229 - val_acc: 0.7419
Epoch 3/80
13837/13837 [==============================] - 5s 354us/step - loss: 1.2363 - acc: 0.7393 - val_loss: 1.1509 - val_acc: 0.7458
Epoch 4/80
13837/13837 [==============================] - 5s 353us/step - loss: 1.0414 - acc: 0.7640 - val_loss: 0.9811 - val_acc: 0.7594
Epoch 5/80
13837/13837 [==============================] - 5s 370us/step - loss: 0.9705 - acc: 0.7703 - val_loss: 1.0262 - val_acc: 0.7620
Epoch 6/80
13837/13837 [==============================] - 5s 348us/step - loss: 0.8928 - acc: 0.7737 - val_loss: 0.8011 - val_acc: 0.7919
Epoch 7/80
13837/13837 [====

Epoch 56/80
13837/13837 [==============================] - 5s 390us/step - loss: 0.3166 - acc: 0.9077 - val_loss: 0.5686 - val_acc: 0.8420
Epoch 57/80
13837/13837 [==============================] - 5s 387us/step - loss: 0.3102 - acc: 0.9066 - val_loss: 0.5639 - val_acc: 0.8420
Epoch 58/80
13837/13837 [==============================] - 7s 540us/step - loss: 0.2886 - acc: 0.9131 - val_loss: 0.5895 - val_acc: 0.8394
Epoch 59/80
13837/13837 [==============================] - 7s 494us/step - loss: 0.2659 - acc: 0.9196 - val_loss: 0.5730 - val_acc: 0.8479
Epoch 60/80
13837/13837 [==============================] - 6s 437us/step - loss: 0.2594 - acc: 0.9212 - val_loss: 0.6052 - val_acc: 0.8485
Epoch 61/80
13837/13837 [==============================] - 6s 424us/step - loss: 0.2501 - acc: 0.9240 - val_loss: 0.5720 - val_acc: 0.8433
Epoch 62/80
13837/13837 [==============================] - 6s 432us/step - loss: 0.2418 - acc: 0.9290 - val_loss: 0.6007 - val_acc: 0.8362
Epoch 63/80
13837/13837 [==

In [13]:
model.evaluate([X_test_a, X_test_b], y_test)

1538/1538 [==============================] - 0s 145us/step


[0.7517802207615658, 0.8335500650195059]